In [ ]:
!pip install nmslib

In [ ]:
!pip install LibRecommender

In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import SVDpp, RNN4Rec  # pure data, algorithm SVD++
from libreco.evaluation import evaluate

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
data["label"] = 1 
# data = pd.read_csv(MOVIE_PATH, sep="::", names=["user", "item", "label", "time"])
metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]
# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
eval_data.build_negative_samples(data_info) 
test_data.build_negative_samples(data_info)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

model = RNN4Rec(task="ranking", data_info=data_info, embed_size=16, n_epochs=3, lr=0.001,
              reg=None, batch_size=256)
model.fit(train_data, verbose=2, eval_data=eval_data, metrics=metrics)
model.init_knn(approximate=True, sim_type="cosine")
model.search_knn_users(user=1, k=3)
model.search_knn_items(item=2, k=3)

# do final evaluation on test data
print("evaluate_result: ", evaluate(model=model, data=test_data,
                                    metrics=metrics))
# predict preference of user 2211 to item 110
print("prediction: ", model.predict(user=2211, item=110))
# recommend 7 items for user 2211
print("recommendation: ", model.recommend_user(user=2211, n_rec=7))

# cold-start prediction
print("cold prediction: ", model.predict(user="ccc", item="not item",
                                         cold_start="average"))
# cold-start recommendation
print("cold recommendation: ", model.recommend_user(user="are we good?",
                                                    n_rec=7,
                                                    cold_start="popular"))


random neg item sampling elapsed: 0.154s
random neg item sampling elapsed: 0.013s
random neg item sampling elapsed: 0.010s
n_users: 610, n_items: 8237, data sparsity: 1.6056 %
total params: 274,221 | embedding params: 271,917 | network params: 2,304
Training start time: 2022-10-30 20:09:54


train: 100%|██████████| 631/631 [00:24<00:00, 26.01it/s]


Epoch 1 elapsed: 24.327s
	 train_loss: 0.5225


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 2673.19it/s]


	 eval log_loss: 0.5082
	 eval roc_auc: 0.8311
	 eval precision@10: 0.0298
	 eval recall@10: 0.0307
	 eval map@10: 0.0692
	 eval ndcg@10: 0.1030


...

Epoch 3 elapsed: 15.402s
	 train_loss: 0.4054


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 2622.33it/s]


	 eval log_loss: 0.5470
	 eval roc_auc: 0.8065
	 eval precision@10: 0.0289
	 eval recall@10: 0.0317
	 eval map@10: 0.0699
	 eval ndcg@10: 0.1037
using approximate searching mode...


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 2907.25it/s]


evaluate_result:  {'loss': 0.5856958662919252, 'roc_auc': 0.784601032029443, 'precision': 0.022697368421052633, 'recall': 0.020756789378215065, 'map': 0.06326330148287385, 'ndcg': 0.08798167775564955}
Detect 1 unknown interaction(s), position: [0]
prediction:  [0.9796386]
detect unknown user: 2211
recommendation:  [(260, 0.98528856), (356, 0.9848156), (3578, 0.9810573), (110, 0.9796386), (457, 0.9745798), (1210, 0.97250855), (593, 0.9702036)]
Detect 1 unknown interaction(s), position: [0]
cold prediction:  [0.19510597]
detect unknown user: are we good?
cold recommendation:  [356, 318, 296, 2571, 593, 260, 110]
